In [1]:
import pandas as pd

In [2]:
dataset_path = "spotify_millsongdata.csv"
df = pd.read_csv(dataset_path)

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.shape

(57650, 4)

In [5]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
df=df.drop('link',axis=1).reset_index(drop=True)

In [7]:
df['text'][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [8]:
df=df.sample(5000)

In [9]:
df.shape

(5000, 3)

In [10]:
df['text'].str.lower().replace('[^a-z0-9]', ' ').head()

9183     oh ...  \r\n  \r\nnow women is losers  \r\nnow...
30178    i've been so many places in my life and time  ...
48427    oh, my girl sitting in the sky  \r\ngo buy can...
52462    i woke up this morning  \r\nwishing you were h...
26493    people can take everything away from you  \r\n...
Name: text, dtype: object

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [13]:
# import nltk
# nltk.download('punkt')  
# nltk.download('punkt_tab')  

# from nltk.tokenize import word_tokenize
# def tokenization(text):
#     return word_tokenize(text)
# df['text'] = df['text'].apply(lambda x: tokenization(x))


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Join the tokenized words back into a string
df['text'] = df['text'].apply(lambda x: ' '.join(x))

# Initialize the TfidfVectorizer
tfidvector = TfidfVectorizer(analyzer='word', stop_words='english')

# Fit and transform the text data
matrix = tfidvector.fit_transform(df['text'])

# Compute cosine similarity
similarity = cosine_similarity(matrix)


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [80]:

similarity[0]

array([1.        , 0.03130795, 0.00251394, ..., 0.00703877, 0.        ,
       0.        ])

In [81]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [82]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [83]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))